In [ ]:
import os
seed = 69
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
# os.environ["TF_DETERMINISTIC_OPS"] = "1" # fix
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
os.environ["PYTHONHASHSEED"] = str(seed)
import pandas as pd
import numpy as np
import random
import time
import gc
import joblib
import pickle
import subprocess
import logging
import datetime as dtm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from xgboost.sklearn import XGBRegressor
from catboost import CatBoostRegressor
# from autogluon.tabular import TabularPredictor
from sklearn.metrics import accuracy_score
from scipy import stats
import tensorflow as tf
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args
from tensorflow.keras.layers import Lambda
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# set seed
tf.random.set_seed(seed)
random.seed(seed)
np.random.seed(seed)


In [ ]:
def weighted_average(a):
    w = []
    n = len(a)
    for j in range(1, n + 1):
        j = 2 if j == 1 else j
        w.append(1 / (2**(n + 1 - j)))
    return np.average(a, weights = w)


            
logging.basicConfig(
    format=
    "%(asctime)s - %(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s",
    datefmt="[%Y-%m-%d %H:%M:%S]",
    level=logging.INFO)
logger = logging.getLogger(__name__)
def reduce_mem(df, use_float16=False):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    tm_cols = df.select_dtypes('datetime').columns
    for col in df.columns:
        if col in tm_cols:
            continue
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(
                        np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(
                        np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(
                        np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(
                        np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if use_float16 and c_min > np.finfo(
                        np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(
                        np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(
        start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def mkdirs(dir2make):
    if isinstance(dir2make, list):
        for i_dir in dir2make:
            if not os.path.exists(i_dir):
                os.makedirs(i_dir)
    elif isinstance(dir2make, str):
        if not os.path.exists(dir2make):
            os.makedirs(dir2make)
    else:
        raise ValueError("dir2make should be string or list type.")

BASE_DIR_cwd = os.getcwd()
BASE_DIR = BASE_DIR_cwd


class Cache():
    @staticmethod
    def cache_data(data, nm_marker=None, dt_format='%Y%m%d_%Hh'):
        mkdirs(os.path.join(BASE_DIR, 'cached_data'))
        name_ = dtm.datetime.now().strftime(dt_format)
        if nm_marker is not None:
            name_ = nm_marker
        path_ = os.path.join(BASE_DIR, f'cached_data/CACHE_{name_}.pkl')
        with open(path_, 'wb') as file:
            pickle.dump(data, file, protocol=4)
        logger.info(f'Cache Successfully! File name: {path_}')

    @staticmethod
    def reload_cache(file_nm,
                     pure_nm=False,
                     base_dir=None,
                     prefix='CACHE_',
                     postfix='.pkl'):
        if pure_nm:
            file_nm = prefix + file_nm + postfix
        if base_dir is None:
            base_dir = os.path.join(BASE_DIR, 'cached_data')
        file_path = os.path.join(base_dir, file_nm)
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
        logger.info(f'Successfully Reload: {file_path}')
        return data

In [ ]:
n_features = 300
features = [f'f_{i}' for i in range(n_features)]

data_types_dict = {
    'time_id': 'int32',
    'investment_id': 'int32',
    "target": 'float16',
}

for f in features:
    data_types_dict[f] = 'float16'

In [ ]:
%%time
path = "/kaggle/input/ubiquant-market-prediction"
train = pd.read_pickle('../input/ubiquant-market-prediction-half-precision-pickle/train.pkl')
with open('../input/ubiquant-time-id-mapping/mapping_dict_all_2014_2022.pkl', 'rb') as file:
    mapping_dict = pickle.load(file)

In [ ]:
# add time feature
import datetime as dt
df_time = train[["time_id"]].drop_duplicates()
df_time["trade_date"] = df_time["time_id"].map(dict(zip(mapping_dict.values(),mapping_dict.keys())))
df_time["trade_date"] = df_time["trade_date"].map(lambda x:dt.datetime.strptime(str(x),"%Y%m%d"))

In [ ]:
import calendar

def week_of_month(tgtdate):
    days_this_month = calendar.mdays[tgtdate.month]
    for i in range(1, days_this_month):
        d = dt.datetime(tgtdate.year, tgtdate.month, i)
        if d.day - d.weekday() > 0:
            startdate = d
            break
    # now we canuse the modulo 7 appraoch
    return (tgtdate - startdate).days //7 + 1

In [ ]:
def get_time_feature(df_time):
    df_time["trade_date_year"]=df_time["trade_date"].map(lambda x:x.year)
    df_time["trade_date_month"]=df_time["trade_date"].map(lambda x:x.month)
    df_time["trade_date_weekday"]=df_time["trade_date"].map(lambda x:x.isoweekday())
    df_time["trade_date_day"]=df_time["trade_date"].map(lambda x:x.day)
    df_time["trade_date_year_rank"]=df_time["time_id"].groupby(df_time["trade_date_year"]).rank()
    df_time["week_of_month"] = df_time["trade_date"].map(lambda x:week_of_month(x))
    return df_time
df_time = get_time_feature(df_time)

In [ ]:
train = train.merge(df_time[["time_id","trade_date_year","trade_date_month","trade_date_weekday","trade_date_day","trade_date_year_rank","week_of_month"]]
                    ,on="time_id",how="left")

In [ ]:
from tqdm import tqdm
class TimeCache:
    def __init__(self):
        self.lasttime = {}  # 上次出现的time_id
        self.lastfelist = {}
        
    def set_time_value(self, key, value):
        self.lasttime[key] = value

    def get_time_value(self, key, value_now):
        if key in self.lasttime:
            value_last = self.lasttime[key]
            self.lasttime[key] = value_now
            return value_last
        else:
            self.lasttime[key] = value_now
            return None
        
    def get_fe_value(self, key, fe, value_now):
        if key in self.lastfelist:
            value_last = self.lastfelist[key][fe]
            self.lastfelist[key][fe] = value_now
            return value_last
        else:
            return value_now
        
    def set_fe_value(self, key, fe, value_now):
        if key in self.lastfelist:
            self.lastfelist[key][fe]=value_now
        else:
            self.lastfelist[key]={fe: value_now}
        
        
        
tc = TimeCache()
# last occur time
last_times = train.groupby("investment_id")["time_id"].apply(lambda x:x.iloc[-1]).rename("time_id").reset_index()
# add import fes lag
n_features = 100
felist=[f'f_{i+100}' for i in range(n_features)]
for i, values in tqdm(last_times.iterrows()):
    last_time = int(values["time_id"])
    key = int(values["investment_id"])
    tc.set_time_value(key, last_time)
last_df = train.groupby("investment_id").apply(lambda x: x.tail(1)).reset_index(drop=True)
for i, values in tqdm(last_df.iterrows()):
    key = int(values["investment_id"])
    for fe in felist:
        last_value = values[fe]
        tc.set_fe_value(key, fe, last_value)
# train
train["lagging_occur_time_diff"] = train.groupby("investment_id")["time_id"].diff()-1 # 最低0天 上次出现time_id
max_fill = train["lagging_occur_time_diff"].max()+1
train["lagging_occur_time_diff"] = train["lagging_occur_time_diff"].fillna(max_fill)
for fe in tqdm(felist):
    train[f"lagging_{fe}_rate"] = train.groupby("investment_id")[fe].diff() # 上次出现value
    train[f"lagging_{fe}_rate"] = train[f"lagging_{fe}_rate"].fillna(0).astype(np.float16)

In [ ]:
del last_times, last_df
joblib.dump(tc,"tc.pkl")
del tc
gc.collect()

In [ ]:
def modify_features(x, mode="train", col_status={}):
    # time
    for col in ["trade_date_year","trade_date_month","trade_date_weekday","trade_date_day","trade_date_year_rank",
               "lagging_occur_time_diff","week_of_month"]+[f'lagging_f_{i+100}_rate' for i in range(n_features)]:
        if mode=="train":
            col_status[col]=(x[col].min(), x[col].max())
        x[col] = (x[col]-col_status[col][0])/(col_status[col][1]-col_status[col][0])
        x[col] = x[col].astype(np.float16)
    return x, col_status
train, col_status = modify_features(train, mode="train")
features += ["trade_date_year","trade_date_month","trade_date_weekday","trade_date_day","trade_date_year_rank",
            "lagging_occur_time_diff","week_of_month"]
features += [f'lagging_f_{i+100}_rate' for i in range(n_features)]
train = train.loc[~train["investment_id"].isin([85, 905, 2558, 3662, 2800, 1415])].reset_index(drop=True)

In [ ]:
gc.collect()


In [ ]:
investment_id = train.pop("investment_id")
investment_id.head()
_ = train.pop("time_id")
y = train.pop("target")
y.mean()

In [ ]:
# Making Tesorflow dataset
def preprocess(X, y):
    return X, y

def make_dataset(feature, y, batch_size=1024, mode="train"):
    ds = tf.data.Dataset.from_tensor_slices(((feature), (y, y)))
    ds = ds.map(preprocess)
    if mode == "train":
        ds = ds.shuffle(4096)
    ds = ds.batch(batch_size)
    return ds

In [ ]:
class CosineLayer():
 
    def __call__(self, x1, x2):
 
        def _cosine(x):
            dot1 = K.batch_dot(x[0], x[1], axes=1)
            dot2 = K.batch_dot(x[0], x[0], axes=1)
            dot3 = K.batch_dot(x[1], x[1], axes=1)
            max_ = K.maximum(K.sqrt(dot2 * dot3), K.epsilon())
            return dot1 / max_
 
        output_shape = (1,)
        value = Lambda(
            _cosine,
            output_shape=output_shape)([x1, x2])
        return value

def my_cor(y_true, y_pred):
    y_true = y_true - K.mean(y_true)
    y_pred = y_pred - K.mean(y_pred)
    t_m1 = K.sqrt(K.sum(y_pred ** 2))# + 0.00001
    t_m2 = K.sqrt(K.sum(y_true ** 2))# + 0.00001

    correlation = K.sum(y_pred*y_true) / (t_m1 * t_m2 + 0.00001)
    return correlation

def spearmancorrelationloss(temp=1.0):
    # 有用
    def loss_fn(y_true, y_pred):
        cosine = CosineLayer()
        similarity = temp - cosine(tf.reshape(y_true,(1, -1)), tf.reshape(y_pred,(1, -1)))
        return similarity
    return loss_fn

def get_model():
    features_inputs = tf.keras.Input((len(features), ), dtype=tf.float16)

    
    feature_x = tf.keras.layers.Dense(512, activation='swish')(features_inputs)
    feature_x = tf.keras.layers.Dropout(0.5)(feature_x)
    feature_x = tf.keras.layers.Dense(256, activation='swish')(feature_x)
    feature_x = tf.keras.layers.Dense(256, activation='swish')(feature_x)
    feature_x = tf.keras.layers.Dense(256, activation='swish')(feature_x)
    
    # x = tf.keras.layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = tf.keras.layers.Dense(256, activation='swish', kernel_regularizer="l2")(feature_x)
    x = tf.keras.layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    x = tf.keras.layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)



    output_0 = tf.keras.layers.Dense(1,name = 'regression')(x)
    output_1 = tf.keras.layers.Dense(1,name = 'cor')(x)
    model = tf.keras.Model(inputs=[features_inputs], outputs=[output_0, output_1])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss={'regression': tf.keras.losses.MeanSquaredError(),
                                                             'cor':spearmancorrelationloss(temp=1.0)}, 
                  metrics = {'regression': tf.keras.metrics.MeanAbsoluteError(name = 'MAE'), 
                             'cor': my_cor
                            }, )
    return model

In [ ]:
gc.collect()

In [ ]:
%%time
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(5, shuffle=False)
for index, (train_indices, valid_indices) in enumerate(kfold.split(train, investment_id)):
    print(f"start save model data fold {index}")
    X_train= train[features].iloc[train_indices]
    Cache.cache_data(X_train, f'X_train_{index}')
    del X_train
    gc.collect()
    X_val = train[features].iloc[valid_indices]
    Cache.cache_data(X_val, f'X_val_{index}')
    del X_val
    gc.collect()
    investment_id_train = investment_id[train_indices]
    Cache.cache_data(investment_id_train, f'investment_id_train_{index}')
    del investment_id_train
    gc.collect()
    y_train = y.iloc[train_indices]
    Cache.cache_data(y_train, f'y_train_{index}')
    del y_train
    gc.collect()
    y_val = y.iloc[valid_indices]
    Cache.cache_data(y_val, f'y_val_{index}')
    del y_val
    gc.collect()
    investment_id_val = investment_id[valid_indices]
    Cache.cache_data(investment_id_val, f'investment_id_val_{index}')
    del investment_id_val
    gc.collect()

In [ ]:
gc.collect()

In [ ]:
del train,y,investment_id,df_time
gc.collect()

In [ ]:
def get_callbacks(index):
    early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_cor_my_cor', patience=4, min_delta = 1e-4, mode = 'max', 
                           baseline = None, restore_best_weights = True, verbose = 0)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(f"model_0159_{index}", save_best_only=True)
    callbacks = [early_stop_callback,  checkpoint]
    return callbacks 

In [ ]:
gc.collect()

In [ ]:
%%time
models = []
scores = []
for index in range(5):
    X_train = Cache.reload_cache(f"CACHE_X_train_{index}.pkl")
    y_train = Cache.reload_cache(f"CACHE_y_train_{index}.pkl")
    investment_id_train = Cache.reload_cache(f"CACHE_investment_id_train_{index}.pkl")
    train_ds = make_dataset(X_train, y_train)
    del X_train, y_train, investment_id_train
    gc.collect()
    X_val = Cache.reload_cache(f"CACHE_X_val_{index}.pkl")
    y_val = Cache.reload_cache(f"CACHE_y_val_{index}.pkl")
    investment_id_val = Cache.reload_cache(f"CACHE_investment_id_val_{index}.pkl")
    valid_ds = make_dataset(X_val, y_val, mode="valid")
    del X_val, investment_id_val
    gc.collect()
    model = get_model()
    early_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_cor_my_cor', patience=4,min_delta = 1e-4, mode = 'max', 
                           baseline = None, restore_best_weights = True, verbose = 0)
    model.fit(train_ds, epochs=50, validation_data=valid_ds, callbacks=get_callbacks(index))
    del train_ds
    gc.collect()
    models.append(model)
    y_hats = model.predict(valid_ds)
    y_hats = (y_hats[0].ravel() *0.8 +y_hats[1].ravel()*0.2)
    pearson_score = stats.pearsonr(y_hats, y_val.values)[0]
    print('Pearson:', pearson_score)
    scores.append(pearson_score)
    del y_hats, y_val
    del valid_ds
    K.clear_session()
    del model
    gc.collect()
print("Valid Scores:", scores)
print("Valid Scores Mean:", np.mean(scores))

In [ ]:
def preprocess_test(feature):
    return (feature), 0

def make_test_dataset(feature, batch_size=1024):
    ds = tf.data.Dataset.from_tensor_slices(((feature)))
    ds = ds.map(preprocess_test)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds

weights = [0.2,0.2,0.2,0.2,0.2]
def inference(models, ds):
    y_preds = []
    for index, model in enumerate(models):
        res = model.predict(ds)
        y_pred = res[0]*0.8+res[1]*0.2
        y_preds.append(y_pred * weights[index])
    return np.sum(y_preds, axis=0)

In [ ]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test() 
tc = joblib.load("tc.pkl")
for (test_df, sample_prediction_df) in iter_test:
    test_df["time_id"]=test_df["row_id"].map(lambda x:int(x.split("_")[0]))# make time_id
    df_time = test_df[["time_id"]].drop_duplicates()
    df_time["trade_date"] = df_time["time_id"].map(dict(zip(mapping_dict.values(),mapping_dict.keys())))
    df_time["trade_date"] = df_time["trade_date"].map(lambda x:dt.datetime.strptime(str(x),"%Y%m%d"))
    df_time = get_time_feature(df_time)
    test_df["lagging_occur_time_diff"]=test_df.apply(lambda x: tc.get_time_value(x["investment_id"], x["time_id"]),axis=1)
    test_df["lagging_occur_time_diff"]=test_df["time_id"]-test_df["lagging_occur_time_diff"]
    test_df["lagging_occur_time_diff"]=test_df["lagging_occur_time_diff"].fillna(max_fill)
    test_df = test_df.merge(df_time[["time_id","trade_date_year","trade_date_month","trade_date_weekday","trade_date_day","trade_date_year_rank","week_of_month"]]
                    ,on="time_id",how="left")
    for fe in felist:
        test_df[f"lagging_{fe}_rate"] = test_df.apply(lambda x: tc.get_fe_value(x["investment_id"], fe, x[fe]),axis=1)
        test_df[f"lagging_{fe}_rate"] = test_df[fe] - test_df[f"lagging_{fe}_rate"] 
    test_df, _ = modify_features(test_df, mode="test", col_status=col_status)
    ds = make_test_dataset(test_df[features])
    sample_prediction_df['target'] = inference(models, ds)
    env.predict(sample_prediction_df) 